In [190]:
import pandas as pd
import numpy as np
import matplotlib


address1 = r'C:\Users\lucme\Desktop\File Transfer\Desktop\Folders - Desktop\Track and Timetable Data from Treno for RER\Track Occupation Data\Union.csv'
occupation_table = pd.read_csv(address1, header=0, sep = ';')
occupation_table.drop(occupation_table.columns[len(occupation_table.columns)-1], axis=1, inplace=True)

occupation_table.insert(column = 'Order', loc = 0, value = occupation_table.index)

occupation_table

,Order,Station,Trains,Origins,Arrival times,Track,Departure times,Destinations,Corresponding trains
0,0,Union Station,E4201,Mimico,04:46:00,UPE,04:55:00,UP Airport,4201
1,1,Union Station,E4203,Mimico,05:01:00,UPE,05:10:00,UP Airport,4203
2,2,Union Station,E4001,Mimico,05:14:00,UPE,05:30:00,UP Airport,4001
3,3,Union Station,E4003,Mimico,05:30:00,UPE,05:45:00,UP Airport,4003
4,4,Union Station,4002,UP Airport,05:52:00,UPE,06:00:00,UP Airport,4005
...,...,...,...,...,...,...,...,...,...
381,381,Union Station,1224,Aldershot,17:35:00,14,17:40:00,Oshawa,9224
382,382,Union Station,6232,Aurora,21:33:00,14,22:15:00,Mount Joy,7436
383,383,Union Station,E7738,Mount Pleasant,22:32:00,14,23:15:00,Lincolnville,7738
384,384,Union Station,7537,Mount Joy,23:27:00,14,23:30:00,Mimico,E7537W


In [191]:
address2 = r'C:\Users\lucme\Desktop\File Transfer\Desktop\Folders - Desktop\Track and Timetable Data from Treno for RER\Track Occupation Data\StationsLineList.csv'
stationslist = pd.read_csv(address2, header=0)
stationslist_origins = stationslist.copy()
stationslist_origins.columns = ['Line','Origins']

stationslist_destinations = stationslist.copy()
stationslist_destinations.columns = ['Line','Destinations']

In [192]:
stationslist_destinations[stationslist_destinations['Line'] == 'UP Express']

,Line,Destinations
34,UP Express,UP Airport


In [193]:
address3 = r'C:\Users\lucme\Desktop\File Transfer\Desktop\Folders - Desktop\Track and Timetable Data from Treno for RER\Track Occupation Data\USCR_Routes.csv'
routeslist = pd.read_csv(address3, header=0, delimiter='\t', encoding='utf16')

lineslist = routeslist.Line.str.split(pat="/",expand=True)

routeslist=pd.concat([routeslist, lineslist], axis=1, sort=False)
routeslist = routeslist.drop(['At_Node','Sequence','Line','Train Types'],axis=1).drop_duplicates(subset=['Route'])
keepcols = routeslist.columns[:4].copy().tolist()+['Line']

zero = routeslist.copy().drop([1,2],axis=1)
one = routeslist.copy().drop([0,2],axis=1)
two = routeslist.copy().drop([0,1],axis=1)

zero.columns = one.columns = two.columns = keepcols
routeslist = pd.concat([zero,one,two],axis=0)
inbound_routes = routeslist[routeslist.Direction == 'Inbound']
outbound_routes = routeslist[routeslist.Direction == 'Outbound']

inbound_routes=inbound_routes.groupby(['Line','Target'])['Route'].apply(list).to_frame().reset_index()

outbound_routes=outbound_routes.groupby(['Line','Source'])['Route'].apply(list).to_frame().reset_index()
inbound_routes.columns = ['Inbound Line','Track','Inbound Routes']
outbound_routes.columns = ['Outbound Line','Track','Outbound Routes']

In [194]:
inbound_routes

,Inbound Line,Track,Inbound Routes
0,Barrie,1,[BARKITCH_IN_1]
1,Barrie,11,"[BARKITCH_IN_10, BARKITCH_IN_11]"
2,Barrie,12,[BARKITCH_IN_8]
3,Barrie,14,[BARKITCH_IN_9]
4,Barrie,2,[BARKITCH_IN_2]
...,...,...,...
91,VIA,10,[LAKE_W_VIANYC_IN_3]
92,VIA,3,[RICHILL_IN_2]
93,VIA,5,[RICHILL_IN_3]
94,VIA,8,"[LAKE_W_VIANYC_IN_1, RICHILL_IN_4]"


In [195]:
outbound_routes[outbound_routes['Outbound Line'] == 'UP Express']

,Outbound Line,Track,Outbound Routes
64,UP Express,1,"[UPE_OUT_3, UPE_OUT_4]"
65,UP Express,UPE,"[UPE_OUT_1, UPE_OUT_2]"


In [196]:
#inbound line
occupation_table = pd.merge(occupation_table, stationslist_origins, how = 'left', on = 'Origins')
#outbound line
occupation_table = pd.merge(occupation_table, stationslist_destinations, how = 'left', on = 'Destinations')
occupation_table


,Order,Station,Trains,Origins,Arrival times,Track,Departure times,Destinations,Corresponding trains,Line_x,Line_y
0,0,Union Station,E4201,Mimico,04:46:00,UPE,04:55:00,UP Airport,4201,Lakeshore West,UP Express
1,1,Union Station,E4203,Mimico,05:01:00,UPE,05:10:00,UP Airport,4203,Lakeshore West,UP Express
2,2,Union Station,E4001,Mimico,05:14:00,UPE,05:30:00,UP Airport,4001,Lakeshore West,UP Express
3,3,Union Station,E4003,Mimico,05:30:00,UPE,05:45:00,UP Airport,4003,Lakeshore West,UP Express
4,4,Union Station,4002,UP Airport,05:52:00,UPE,06:00:00,UP Airport,4005,UP Express,UP Express
...,...,...,...,...,...,...,...,...,...,...,...
381,381,Union Station,1224,Aldershot,17:35:00,14,17:40:00,Oshawa,9224,Lakeshore West,Lakeshore East
382,382,Union Station,6232,Aurora,21:33:00,14,22:15:00,Mount Joy,7436,Barrie,Stouffville
383,383,Union Station,E7738,Mount Pleasant,22:32:00,14,23:15:00,Lincolnville,7738,Kitchener,Stouffville
384,384,Union Station,7537,Mount Joy,23:27:00,14,23:30:00,Mimico,E7537W,Stouffville,Lakeshore West


In [197]:
occupation_table[occupation_table['Destinations'] == 'Don Yard Jn']

,Order,Station,Trains,Origins,Arrival times,Track,Departure times,Destinations,Corresponding trains,Line_x,Line_y
104,104,Union Station,7903,Lincolnville,07:08:00,2,07:13:00,Don Yard Jn,E8122,Stouffville,None
208,208,Union Station,6804,Allandale Waterfront,08:19:00,5,08:24:00,Don Yard Jn,E6804D,Barrie,None
228,228,Union Station,3902,Kitchener,07:43:00,6,07:48:00,Don Yard Jn,E3902D,Kitchener,None
230,230,Union Station,2718,Milton,08:27:00,6,08:32:00,Don Yard Jn,E2718D,Milton,None
240,240,Union Station,2716,Milton,08:15:00,7,08:20:00,Don Yard Jn,E2716D,Milton,None
241,241,Union Station,6904,Allandale Waterfront,08:33:00,7,08:38:00,Don Yard Jn,E6904D,Barrie,None
290,290,Union Station,2712,Milton,07:45:00,12,07:50:00,Don Yard Jn,E2712D,Milton,None
291,291,Union Station,3606,Georgetown,08:18:00,12,08:25:00,Don Yard Jn,E3606D,Kitchener,None
298,298,Union Station,6216,Aurora,13:33:00,12,13:35:00,Don Yard Jn,E6216D,Barrie,None
318,318,Union Station,1852,Hamilton,07:00:00,13,07:05:00,Don Yard Jn,E1852D,Lakeshore West,None


In [198]:
columns = occupation_table.columns.tolist()
columns[1] = 'Station'
columns[-2] = 'Inbound Line'
columns[-1] = 'Outbound Line'
occupation_table.columns = columns
occupation_table

,Order,Station,Trains,Origins,Arrival times,Track,Departure times,Destinations,Corresponding trains,Inbound Line,Outbound Line
0,0,Union Station,E4201,Mimico,04:46:00,UPE,04:55:00,UP Airport,4201,Lakeshore West,UP Express
1,1,Union Station,E4203,Mimico,05:01:00,UPE,05:10:00,UP Airport,4203,Lakeshore West,UP Express
2,2,Union Station,E4001,Mimico,05:14:00,UPE,05:30:00,UP Airport,4001,Lakeshore West,UP Express
3,3,Union Station,E4003,Mimico,05:30:00,UPE,05:45:00,UP Airport,4003,Lakeshore West,UP Express
4,4,Union Station,4002,UP Airport,05:52:00,UPE,06:00:00,UP Airport,4005,UP Express,UP Express
...,...,...,...,...,...,...,...,...,...,...,...
381,381,Union Station,1224,Aldershot,17:35:00,14,17:40:00,Oshawa,9224,Lakeshore West,Lakeshore East
382,382,Union Station,6232,Aurora,21:33:00,14,22:15:00,Mount Joy,7436,Barrie,Stouffville
383,383,Union Station,E7738,Mount Pleasant,22:32:00,14,23:15:00,Lincolnville,7738,Kitchener,Stouffville
384,384,Union Station,7537,Mount Joy,23:27:00,14,23:30:00,Mimico,E7537W,Stouffville,Lakeshore West


In [199]:
occupation_table = pd.merge( occupation_table,inbound_routes,how = 'left', on=['Inbound Line','Track'])
occupation_table = pd.merge( occupation_table,outbound_routes,how = 'left', on=['Outbound Line','Track'])
occupation_table.fillna('None')



,Order,Station,Trains,Origins,Arrival times,Track,Departure times,Destinations,Corresponding trains,Inbound Line,Outbound Line,Inbound Routes,Outbound Routes
0,0,Union Station,E4201,Mimico,04:46:00,UPE,04:55:00,UP Airport,4201,Lakeshore West,UP Express,[LAKE_W_IN_1],"[UPE_OUT_1, UPE_OUT_2]"
1,1,Union Station,E4203,Mimico,05:01:00,UPE,05:10:00,UP Airport,4203,Lakeshore West,UP Express,[LAKE_W_IN_1],"[UPE_OUT_1, UPE_OUT_2]"
2,2,Union Station,E4001,Mimico,05:14:00,UPE,05:30:00,UP Airport,4001,Lakeshore West,UP Express,[LAKE_W_IN_1],"[UPE_OUT_1, UPE_OUT_2]"
3,3,Union Station,E4003,Mimico,05:30:00,UPE,05:45:00,UP Airport,4003,Lakeshore West,UP Express,[LAKE_W_IN_1],"[UPE_OUT_1, UPE_OUT_2]"
4,4,Union Station,4002,UP Airport,05:52:00,UPE,06:00:00,UP Airport,4005,UP Express,UP Express,[UPEMILTKITCH_IN_1],"[UPE_OUT_1, UPE_OUT_2]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,381,Union Station,1224,Aldershot,17:35:00,14,17:40:00,Oshawa,9224,Lakeshore West,Lakeshore East,[LAKE_W_IN_10],"[LAKE_E_EXPSTOUFREG_OUT_11, LAKE_E_EXPSTOUFREG..."
382,382,Union Station,6232,Aurora,21:33:00,14,22:15:00,Mount Joy,7436,Barrie,Stouffville,[BARKITCH_IN_9],"[LAKE_E_STOUFDANF_OUT_10, LAKE_E_STOUFDANF_OUT..."
383,383,Union Station,E7738,Mount Pleasant,22:32:00,14,23:15:00,Lincolnville,7738,Kitchener,Stouffville,[BARKITCH_IN_9],"[LAKE_E_STOUFDANF_OUT_10, LAKE_E_STOUFDANF_OUT..."
384,384,Union Station,7537,Mount Joy,23:27:00,14,23:30:00,Mimico,E7537W,Stouffville,Lakeshore West,[LAKE_E_STOUFDANF_IN_7],None


In [200]:
occupation_table

,Order,Station,Trains,Origins,Arrival times,Track,Departure times,Destinations,Corresponding trains,Inbound Line,Outbound Line,Inbound Routes,Outbound Routes
0,0,Union Station,E4201,Mimico,04:46:00,UPE,04:55:00,UP Airport,4201,Lakeshore West,UP Express,[LAKE_W_IN_1],"[UPE_OUT_1, UPE_OUT_2]"
1,1,Union Station,E4203,Mimico,05:01:00,UPE,05:10:00,UP Airport,4203,Lakeshore West,UP Express,[LAKE_W_IN_1],"[UPE_OUT_1, UPE_OUT_2]"
2,2,Union Station,E4001,Mimico,05:14:00,UPE,05:30:00,UP Airport,4001,Lakeshore West,UP Express,[LAKE_W_IN_1],"[UPE_OUT_1, UPE_OUT_2]"
3,3,Union Station,E4003,Mimico,05:30:00,UPE,05:45:00,UP Airport,4003,Lakeshore West,UP Express,[LAKE_W_IN_1],"[UPE_OUT_1, UPE_OUT_2]"
4,4,Union Station,4002,UP Airport,05:52:00,UPE,06:00:00,UP Airport,4005,UP Express,UP Express,[UPEMILTKITCH_IN_1],"[UPE_OUT_1, UPE_OUT_2]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,381,Union Station,1224,Aldershot,17:35:00,14,17:40:00,Oshawa,9224,Lakeshore West,Lakeshore East,[LAKE_W_IN_10],"[LAKE_E_EXPSTOUFREG_OUT_11, LAKE_E_EXPSTOUFREG..."
382,382,Union Station,6232,Aurora,21:33:00,14,22:15:00,Mount Joy,7436,Barrie,Stouffville,[BARKITCH_IN_9],"[LAKE_E_STOUFDANF_OUT_10, LAKE_E_STOUFDANF_OUT..."
383,383,Union Station,E7738,Mount Pleasant,22:32:00,14,23:15:00,Lincolnville,7738,Kitchener,Stouffville,[BARKITCH_IN_9],"[LAKE_E_STOUFDANF_OUT_10, LAKE_E_STOUFDANF_OUT..."
384,384,Union Station,7537,Mount Joy,23:27:00,14,23:30:00,Mimico,E7537W,Stouffville,Lakeshore West,[LAKE_E_STOUFDANF_IN_7],NaN


In [201]:
inRoutes = occupation_table['Inbound Routes'].apply(pd.Series)
outRoutes = occupation_table['Outbound Routes'].apply(pd.Series)


In [202]:
inRoutes

,0,1,2
0,LAKE_W_IN_1,NaN,NaN
1,LAKE_W_IN_1,NaN,NaN
2,LAKE_W_IN_1,NaN,NaN
3,LAKE_W_IN_1,NaN,NaN
4,UPEMILTKITCH_IN_1,NaN,NaN
...,...,...,...
381,LAKE_W_IN_10,NaN,NaN
382,BARKITCH_IN_9,NaN,NaN
383,BARKITCH_IN_9,NaN,NaN
384,LAKE_E_STOUFDANF_IN_7,NaN,NaN


In [203]:
occupation_table = occupation_table.drop(columns = ['Inbound Routes','Outbound Routes'], axis = 1)
occupation_table

,Order,Station,Trains,Origins,Arrival times,Track,Departure times,Destinations,Corresponding trains,Inbound Line,Outbound Line
0,0,Union Station,E4201,Mimico,04:46:00,UPE,04:55:00,UP Airport,4201,Lakeshore West,UP Express
1,1,Union Station,E4203,Mimico,05:01:00,UPE,05:10:00,UP Airport,4203,Lakeshore West,UP Express
2,2,Union Station,E4001,Mimico,05:14:00,UPE,05:30:00,UP Airport,4001,Lakeshore West,UP Express
3,3,Union Station,E4003,Mimico,05:30:00,UPE,05:45:00,UP Airport,4003,Lakeshore West,UP Express
4,4,Union Station,4002,UP Airport,05:52:00,UPE,06:00:00,UP Airport,4005,UP Express,UP Express
...,...,...,...,...,...,...,...,...,...,...,...
381,381,Union Station,1224,Aldershot,17:35:00,14,17:40:00,Oshawa,9224,Lakeshore West,Lakeshore East
382,382,Union Station,6232,Aurora,21:33:00,14,22:15:00,Mount Joy,7436,Barrie,Stouffville
383,383,Union Station,E7738,Mount Pleasant,22:32:00,14,23:15:00,Lincolnville,7738,Kitchener,Stouffville
384,384,Union Station,7537,Mount Joy,23:27:00,14,23:30:00,Mimico,E7537W,Stouffville,Lakeshore West


In [204]:

occupation_table_out = pd.merge(occupation_table , outRoutes, left_index=True, right_index=True)
occupation_table_out = occupation_table_out.melt(id_vars = occupation_table.columns, value_name='Outbound Route').drop(columns='variable',axis=1).drop_duplicates().sort_values(by='Order')
occupation_table_out

,Order,Station,Trains,Origins,Arrival times,Track,Departure times,Destinations,Corresponding trains,Inbound Line,Outbound Line,Outbound Route
0,0,Union Station,E4201,Mimico,04:46:00,UPE,04:55:00,UP Airport,4201,Lakeshore West,UP Express,UPE_OUT_1
772,0,Union Station,E4201,Mimico,04:46:00,UPE,04:55:00,UP Airport,4201,Lakeshore West,UP Express,NaN
386,0,Union Station,E4201,Mimico,04:46:00,UPE,04:55:00,UP Airport,4201,Lakeshore West,UP Express,UPE_OUT_2
1,1,Union Station,E4203,Mimico,05:01:00,UPE,05:10:00,UP Airport,4203,Lakeshore West,UP Express,UPE_OUT_1
773,1,Union Station,E4203,Mimico,05:01:00,UPE,05:10:00,UP Airport,4203,Lakeshore West,UP Express,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
769,383,Union Station,E7738,Mount Pleasant,22:32:00,14,23:15:00,Lincolnville,7738,Kitchener,Stouffville,LAKE_E_STOUFDANF_OUT_11
383,383,Union Station,E7738,Mount Pleasant,22:32:00,14,23:15:00,Lincolnville,7738,Kitchener,Stouffville,LAKE_E_STOUFDANF_OUT_10
1541,383,Union Station,E7738,Mount Pleasant,22:32:00,14,23:15:00,Lincolnville,7738,Kitchener,Stouffville,NaN
384,384,Union Station,7537,Mount Joy,23:27:00,14,23:30:00,Mimico,E7537W,Stouffville,Lakeshore West,NaN


In [205]:
occupation_table_in = pd.merge(occupation_table , inRoutes, left_index=True, right_index=True)
occupation_table_in = occupation_table_in.melt(id_vars = occupation_table.columns, value_name='Inbound Route').drop(columns='variable',axis=1).drop_duplicates().sort_values(by='Order')
occupation_table_in 

,Order,Station,Trains,Origins,Arrival times,Track,Departure times,Destinations,Corresponding trains,Inbound Line,Outbound Line,Inbound Route
0,0,Union Station,E4201,Mimico,04:46:00,UPE,04:55:00,UP Airport,4201,Lakeshore West,UP Express,LAKE_W_IN_1
386,0,Union Station,E4201,Mimico,04:46:00,UPE,04:55:00,UP Airport,4201,Lakeshore West,UP Express,NaN
387,1,Union Station,E4203,Mimico,05:01:00,UPE,05:10:00,UP Airport,4203,Lakeshore West,UP Express,NaN
1,1,Union Station,E4203,Mimico,05:01:00,UPE,05:10:00,UP Airport,4203,Lakeshore West,UP Express,LAKE_W_IN_1
2,2,Union Station,E4001,Mimico,05:14:00,UPE,05:30:00,UP Airport,4001,Lakeshore West,UP Express,LAKE_W_IN_1
...,...,...,...,...,...,...,...,...,...,...,...,...
383,383,Union Station,E7738,Mount Pleasant,22:32:00,14,23:15:00,Lincolnville,7738,Kitchener,Stouffville,BARKITCH_IN_9
770,384,Union Station,7537,Mount Joy,23:27:00,14,23:30:00,Mimico,E7537W,Stouffville,Lakeshore West,NaN
384,384,Union Station,7537,Mount Joy,23:27:00,14,23:30:00,Mimico,E7537W,Stouffville,Lakeshore West,LAKE_E_STOUFDANF_IN_7
771,385,Union Station,7539,Mount Joy,00:27:00,14,00:30:00,Mimico,E7539W,Stouffville,Lakeshore West,NaN


In [206]:
#occupation_table_in.to_csv(r'C:\Users\LUCME\Desktop\File Transfer\Desktop\Folders - Desktop\Track and Timetable Data from Treno for RER\Output\union_station_track_occupation_inbound_routes_V2.csv', sep='\t',                                    encoding='UTF-16', index = False)
#occupation_table_out.to_csv(r'C:\Users\LUCME\Desktop\File Transfer\Desktop\Folders - Desktop\Track and Timetable Data from Treno for RER\Output\union_station_track_occupation_outbound_routes_V2.csv', sep='\t',                                  encoding='UTF-16', index = False)


In [207]:
def removeBlank(df, groupbycol='Order', droprowscol='Route'):
    
    occupation_table_df_list = []
    for order, group in df.groupby(by=groupbycol):
        if group.shape[0] > 1:
            group.dropna(subset=[droprowscol], inplace=True)
            occupation_table_df_list.append(group)
        else:
            occupation_table_df_list.append(group)

    return occupation_table_df_list

In [209]:
occupation_table_out_df_list = removeBlank(df=occupation_table_out ,groupbycol='Order', droprowscol='Outbound Route')


occupation_table_in_df_list = removeBlank(df=occupation_table_in ,groupbycol='Order', droprowscol='Inbound Route')
        

In [210]:
pd.concat(occupation_table_in_df_list).sort_values(by = 'Order').to_csv(r'C:\Users\LUCME\Desktop\File Transfer\Desktop\Folders - Desktop\Track and Timetable Data from Treno for RER\Output\union_station_track_occupation_inbound_routes_V4.csv', sep='\t',encoding='UTF-16', index = False)

pd.concat(occupation_table_out_df_list).sort_values(by = 'Order').to_csv(r'C:\Users\LUCME\Desktop\File Transfer\Desktop\Folders - Desktop\Track and Timetable Data from Treno for RER\Output\union_station_track_occupation_outbound_routes_V4.csv', sep='\t',encoding='UTF-16', index = False)